`02_prompt_engineering/homework_pe_20260109.ipynb`파일에 작성하세요.

### 1. 과제 개요 (Overview)

- **목표:** 학교생활이나 일상생활에서 겪는 사소한 불편함이나 비효율적인 문제를 발견하고, LLM과 프롬프트 엔지니어링 기술을 활용하여 이를 해결하는 웹 애플리케이션(프로토타입)을 제작한다.
- **핵심 역량:** 문제 해결 능력, 프롬프트 엔지니어링(Prompt Engineering), 기초 AI 애플리케이션 구현 능력.

### 2. 단계별 수행 가이드 (Step-by-Step Guide)

**Step 1: 문제 발굴 및 정의 (Problem Definition)**

- 나의 일상이나 친구들이 겪는 불편함을 관찰한다. 거창한 문제가 아니어도 된다.
- *예시:*
    - "오늘 급식 메뉴를 보고 맛있는 조합인지 아닌지 판단하기 귀찮다."
    - "동아리 공지사항을 쓸 때마다 말투를 다듬는 게 어렵다."
    - "복잡한 가정통신문 내용을 3줄로 요약해서 보고 싶다."
- **과제:** 해결하고자 하는 문제를 한 문장으로 정의하고, 타겟 사용자(나 자신 혹은 친구들)를 설정한다.

**Step 2: 프롬프트 엔지니어링 (Prompt Engineering)**

- LLM이 문제를 해결하도록 지시하는 최적의 프롬프트를 설계한다.
- **필수 포함 요소:**
    - **Role (페르소나):** AI에게 역할 부여 (예: "너는 영양사이자 메뉴 추천 전문가야.")
    - **Context (맥락):** 상황 설명 (예: "고등학생들이 좋아하는 입맛을 기준으로 분석해야 해.")
    - **Task (지시 사항):** 구체적인 수행 작업 (예: "메뉴 리스트를 입력받으면, 10점 만점으로 점수를 매기고 그 이유를 설명해 줘.")
    - **Constraints (제약 조건):** 출력 형식 제한 (예: "반말을 사용하고, 이모지를 3개 이상 포함해.")
- **과제:** 다양한 프롬프트를 테스트해 보고, 가장 결과가 좋은 최종 프롬프트를 선정한다.

**Step 3: 앱 구현 (Implementation)**

- 선정된 프롬프트를 실제 작동하는 앱 형태로 만든다.
- **도구 추천:**
    - GPT Chat Playground
    - Python + Streamlit (간단한 파이썬 코드로 웹 UI 제작)
- **과제:** 사용자가 입력창에 텍스트를 넣으면, LLM이 처리하여 결과를 보여주는 인터페이스를 구현한다.

**Step 4: 테스트 및 개선 (Testing & Iteration)**

- 친구들에게 앱을 써보게 하고 피드백을 받는다.
- LLM이 엉뚱한 대답을 하는 경우(할루시네이션 등), 프롬프트를 수정하여 성능을 개선한다.

### 3. 예시 시나리오 (Example Scenarios)

| **구분** | **앱 이름** | **기능 설명** | **프롬프트 핵심 전략** |
| --- | --- | --- | --- |
| **생활** | **냉장고 파먹기 AI** | 냉장고에 남은 재료만 입력하면 레시피 추천 | "가진 재료만 써야 함", "자취생이 할 수 있는 쉬운 요리" 제약 조건 추가 |
| **학업** | **발표 대본 생성기** | 조사한 자료를 넣으면 발표용 대본으로 변환 | "3분 분량으로 맞춰줘", "청중의 흥미를 끌 수 있는 유머 포함" 지시 |
| **소통** | **'너 T야?' 탐지기** | 카톡 메시지를 입력하면 공감 능력이 부족한지 판별 | "공감형 대화 전문가" 페르소나 부여, 감정 분석 및 수정 제안 |

In [ ]:
# 1. 필요한 라이브러리 설치 (Streamlit, OpenAI)
!pip install -q streamlit openai

# 2. 터널링을 위한 도구 설치 (외부에서 접속할 링크 생성용)
!npm install localtunnel

In [ ]:
import streamlit as st
import os
import json
import datetime
from openai import OpenAI
from dotenv import load_dotenv
from duckduckgo_search import DDGS

# 1. 환경 변수 로드
load_dotenv()

# 2. 페이지 설정
st.set_page_config(page_title="개발자 취업 예측기 Pro", page_icon="🔮", layout="wide")

# 3. Helper 함수: 실시간 시장 동향 검색 (DuckDuckGo 사용 - 무료)
@st.cache_data(ttl=3600 * 24) # 24시간마다 갱신 (하루에 한 번만 검색)
def get_market_trends():
    current_date = datetime.datetime.now().strftime("%Y년 %m월")
    query = f"{current_date} 한국 IT 개발자 채용 시장 동향 전망"
    
    try:
        # 최근 검색 결과 5개를 가져와 요약
        results = DDGS().text(keywords=query, region='kr-kr', max_results=5)
        search_summary = "\n".join([f"- {r['title']}: {r['body']}" for r in results])
        return f"[{current_date} 기준 검색된 시장 동향]\n{search_summary}"
    except Exception as e:
        return f"[{current_date} 기준] 검색 오류 발생. AI의 기본 지식으로 대체합니다."

# 4. 모델 매핑
def get_real_model_name(user_choice):
    if user_choice == "4.1-mini": return "gpt-4o-mini"
    elif user_choice == "4.1": return "gpt-4o"
    return "gpt-4o-mini"

# 5. UI 구성
st.title("🔮 AI 개발자 취업 가능성 예측기 (Live Ver.)")
st.markdown("---")

# 사이드바 설정
with st.sidebar:
    st.header("⚙️ 설정")
    api_key = os.getenv("OPENAI_API_KEY")
    if not api_key:
        api_key = st.text_input("OpenAI API Key", type="password")
    
    model_option = st.selectbox("사용 모델", ["4.1-mini", "4.1"], index=0)

# --- 메인 입력 화면 (객관식 위주 변경) ---
col1, col2 = st.columns([1, 1])

with col1:
    st.subheader("1️⃣ 개인 스펙 입력")
    with st.container(border=True):
        # 직군 선택
        target_role = st.selectbox(
            "희망 직군",
            ["백엔드 개발자", "프론트엔드 개발자", "풀스택 개발자", "AI/머신러닝 엔지니어", "데이터 사이언티스트", "DevOps/클라우드", "모바일 앱 개발자"]
        )
        
        # 학력 및 전공 (객관식)
        education = st.radio("전공 여부", ["전공자 (컴공/SW관련)", "비전공자 (이공계)", "비전공자 (인문/상경/기타)"], horizontal=True)
        degree = st.selectbox("최종 학력", ["고졸", "초대졸", "학사(대졸)", "석사", "박사"])
        
        # 교육 이수 (객관식)
        training = st.multiselect(
            "이수한 교육 과정 (복수 선택 가능)",
            ["K-Digital Training(국비 부트캠프)", "사설 부트캠프(유료)", "온라인 강의 독학", "CS 전공 수업만 수강", "알고리즘/코딩테스트 스터디"]
        )
        
        # 보유 자격증 (객관식)
        certs = st.multiselect(
            "보유 자격증",
            ["정보처리기사/산업기사", "SQLD/SQLP", "AWS/Cloud 관련", "ADsP/ADP", "토익 800+ / 오픽 IM2+"]
        )
        
        # 기술 스택 (객관식 + 직접 입력)
        tech_stack = st.multiselect(
            "자신 있는 기술 스택 (Top 3)",
            ["Python", "Java/Spring", "JavaScript/Node.js", "React/Next.js", "Vue/Nuxt", "C++", "Go", "Docker/K8s", "PyTorch/Tensorflow"]
        )

        # 주관식 (최소화)
        project_exp = st.text_area(
            "프로젝트/협업 경험 (핵심만 서술)",
            placeholder="예: 팀 프로젝트 2회 경험, 캡스톤 디자인 수상, 실제 서비스 배포 및 운영 경험 있음 (사용자 100명)",
            height=100
        )

with col2:
    st.subheader("2️⃣ 현재 사회/시장 환경 (자동 분석)")
    
    # 실시간 검색 로딩
    with st.spinner("최신 채용 동향을 검색하고 있습니다..."):
        market_info = get_market_trends()
    
    # 검색된 내용을 보여주되, 수정 가능하게 함 (사용자가 더 잘 알 수도 있으므로)
    final_market_env = st.text_area(
        "AI가 수집한 현재 시장 상황 (필요 시 수정 가능)",
        value=market_info,
        height=480
    )

# 분석 버튼
st.markdown("---")
analyze_btn = st.button("🚀 취업 가능성 분석 시작", type="primary", use_container_width=True)

# 6. 분석 로직
if analyze_btn:
    if not api_key:
        st.error("API Key를 입력해주세요.")
    else:
        client = OpenAI(api_key=api_key)
        
        # 입력된 객관식 데이터를 텍스트로 정리
        user_profile = f"""
        - 희망 직군: {target_role}
        - 학력/전공: {degree}, {education}
        - 교육 이수: {', '.join(training)}
        - 자격증: {', '.join(certs)}
        - 기술 스택: {', '.join(tech_stack)}
        - 상세 경험: {project_exp}
        """

        system_msg = """
        당신은 한국 최고의 IT 취업 컨설턴트입니다. 
        사용자의 정형화된 스펙과 현재의 실시간 시장 채용 동향(검색 데이터)을 기반으로 분석하십시오.
        
        **필수 출력 포맷(JSON Only):**
        {
            "probability": 숫자(0-100),
            "summary": "한줄 평",
            "analysis": {
                "strengths": ["강점1", "강점2"],
                "weaknesses": ["약점1", "약점2"],
                "market_fit": "시장 상황과 스펙의 적합성 분석"
            },
            "action_plan": ["구체적 행동1", "구체적 행동2", "구체적 행동3"]
        }
        """

        user_prompt = f"""
        [분석 요청]
        1. 사용자 프로필: {user_profile}
        2. 현재 시장 상황: {final_market_env}
        
        위 정보를 바탕으로 냉철하게 합격 확률을 예측하고 JSON으로 답해줘.
        """
        
        with st.spinner("데이터 분석 중..."):
            try:
                response = client.chat.completions.create(
                    model=get_real_model_name(model_option),
                    messages=[
                        {"role": "system", "content": system_msg},
                        {"role": "user", "content": user_prompt}
                    ],
                    response_format={"type": "json_object"}
                )
                
                result = json.loads(response.choices[0].message.content)
                
                # 결과 출력 UI
                st.success("분석 완료!")
                
                # 확률 게이지
                prob = result['probability']
                col_res1, col_res2 = st.columns([1, 3])
                with col_res1:
                    st.metric("예상 합격률", f"{prob}%")
                with col_res2:
                    st.progress(prob / 100)
                    st.write(f"**요약:** {result['summary']}")
                
                # 상세 탭
                tab1, tab2 = st.tabs(["상세 분석", "Action Plan"])
                with tab1:
                    c1, c2 = st.columns(2)
                    with c1:
                        st.markdown("#### ✅ 강점")
                        for s in result['analysis']['strengths']: st.success(f"- {s}")
                    with c2:
                        st.markdown("#### ⚠️ 보완점")
                        for w in result['analysis']['weaknesses']: st.error(f"- {w}")
                    st.info(f"**시장 적합도:** {result['analysis']['market_fit']}")
                    
                with tab2:
                    st.markdown("#### 🏃 당장 실천할 것")
                    for action in result['action_plan']:
                        st.write(f"✔️ {action}")

            except Exception as e:
                st.error(f"오류 발생: {str(e)}")
